Import modules

In [3]:
import boto3
import json
import os
from dotenv import load_dotenv 
import cv2


Get environment variables and create AWS Session

In [ ]:

dotenv_path = os.path.join("..",'.env')
load_dotenv(dotenv_path)

#Connect to AWS Ressource, Credentials are optionals as we are accessing from the same AWS user
client=boto3.client('rekognition',aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME"))
s3 = boto3.client('s3',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME")
)

#Define bucket name to access
bucket_name = os.getenv("BUCKET_NAME")

#Folder of real classroom photos
classroomFolder=os.path.join("..","ImagesRekognition","RealClassroomPhotos")

Take all images of the folder and for each draw detected faces and save it to FacesDetectedInClassroom

In [ ]:

#Cycle that only draws rectangles where there are faces according to Rekognition's algorithm
i=1
boundingBoxDetectedFaces=[]

for file in os.listdir(classroomFolder):
    camera_local_file = os.path.join("..","ImagesRekognition","RealClassroomPhotos",file)
    #convert files to bytes in order to be able to read it

    with open(camera_local_file, 'rb') as image_file:
        camera_file_bytes = image_file.read()
    responseDetection=client.detect_faces( Image={'Bytes': camera_file_bytes})
    # Read the image with OpenCV
    image = cv2.imread(camera_local_file)
    image_height, image_width, _ = image.shape

    boundingBoxDetectedFacesithImage=[]

    # Draw bounding boxes for each detected face
    for face in responseDetection['FaceDetails']:
        bbox = face['BoundingBox']
        left = int(bbox['Left'] * image_width)
        top = int(bbox['Top'] * image_height)
        width = int(bbox['Width'] * image_width)
        height = int(bbox['Height'] * image_height)
        cv2.rectangle(
            image, 
            (left, top), 
            (left + width, top + height), 
            (37, 173, 250), 
            2  
        )
        boundingBoxDetectedFacesithImage.append((left, top, width,height))
    #Each array element represents a sub-array of tuples containing the bonding boxes    
    boundingBoxDetectedFaces.append(boundingBoxDetectedFacesithImage) 
    # Print number of faces detected
    print(f"{len(responseDetection['FaceDetails'])} Faces were detected")

    # Save the annotated image
    output_path =os.path.join("..","ImagesRekognition","FacesDetectedInClassroom",f"faces_detected_{i}.jpeg")
    cv2.imwrite(output_path, image)
    #Increase the photo's index
    i=i+1


In [29]:
for file in os.listdir(classroomFolder):
    if (int(str(file).split(".")[0][-1])>3):
        print(str(file).split(".")[0][-1]) 

In [16]:

def calculate_iou_detection(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    # Extraer coordenadas
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calcular coordenadas de la intersección
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    # Comprobar si hay intersección
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    # Calcular áreas
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    # Calcular unión
    area_union = area_box1 + area_box2 - area_intersect
    
    # Calcular IoU
    iou = area_intersect / area_union
    
    return iou




In [17]:
true=True
groundTruth=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":18,"y":509,"width":31,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":166,"y":487,"width":30,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":122,"y":538,"width":36,"height":54},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":345,"y":485,"width":36,"height":45},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":371,"y":425,"width":30,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":410,"y":583,"width":50,"height":77},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":570,"y":543,"width":51,"height":65},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":807,"y":459,"width":46,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":399,"width":29,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1159,"y":367,"width":24,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1192,"y":332,"width":25,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":909,"y":349,"width":23,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":796,"y":365,"width":26,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":737,"y":403,"width":27,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":715,"y":351,"width":22,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":621,"y":377,"width":26,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":439,"y":369,"width":23,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":611,"y":338,"width":21,"height":23},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":694,"y":331,"width":18,"height":22},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":660,"y":278,"width":19,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":544,"y":428,"width":39,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom2.jpeg152579":{"filename":"classroom2.jpeg","size":152579,"regions":[{"shape_attributes":{"name":"rect","x":26,"y":506,"width":31,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":158,"y":485,"width":32,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":174,"y":537,"width":38,"height":59},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":372,"y":425,"width":29,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":341,"y":481,"width":37,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":442,"y":367,"width":23,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":427,"y":579,"width":53,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":582,"y":546,"width":52,"height":59},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":808,"y":458,"width":47,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1036,"y":394,"width":28,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1126,"y":379,"width":25,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1184,"y":329,"width":29,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":910,"y":349,"width":23,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":806,"y":364,"width":27,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":730,"y":397,"width":28,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":719,"y":352,"width":21,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":619,"y":379,"width":27,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":615,"y":338,"width":22,"height":23},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":694,"y":335,"width":18,"height":17},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":542,"y":429,"width":38,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":660,"y":277,"width":20,"height":22},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom3.jpg155001":{"filename":"classroom3.jpg","size":155001,"regions":[{"shape_attributes":{"name":"rect","x":67,"y":800,"width":68,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":357,"y":718,"width":64,"height":68},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":531,"y":593,"width":54,"height":61},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":603,"y":499,"width":38,"height":46},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":454,"width":30,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":773,"y":542,"width":39,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1024,"y":483,"width":32,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":433,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":411,"width":25,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom4.jpg156291":{"filename":"classroom4.jpg","size":156291,"regions":[{"shape_attributes":{"name":"rect","x":72,"y":800,"width":65,"height":74},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":362,"y":720,"width":62,"height":71},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":541,"width":42,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":551,"y":595,"width":48,"height":56},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":501,"width":36,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":410,"width":26,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1024,"y":481,"width":32,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":433,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":451,"width":28,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom5.jpg156524":{"filename":"classroom5.jpg","size":156524,"regions":[{"shape_attributes":{"name":"rect","x":69,"y":799,"width":67,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":366,"y":722,"width":59,"height":65},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":610,"y":499,"width":33,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":549,"y":589,"width":50,"height":62},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":796,"y":454,"width":26,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":769,"y":539,"width":42,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1020,"y":481,"width":34,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":411,"width":26,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1107,"y":430,"width":26,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom6.jpg153504":{"filename":"classroom6.jpg","size":153504,"regions":[{"shape_attributes":{"name":"rect","x":61,"y":844,"width":66,"height":75},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":216,"y":721,"width":64,"height":71},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":429,"y":700,"width":56,"height":61},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":585,"y":653,"width":46,"height":54},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":611,"width":44,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":973,"y":534,"width":33,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1076,"y":514,"width":29,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":956,"y":469,"width":25,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":903,"y":500,"width":21,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":808,"y":526,"width":23,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":701,"y":531,"width":36,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":575,"y":566,"width":39,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":154,"y":576,"width":34,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":192,"y":550,"width":24,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":289,"y":518,"width":25,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":241,"y":579,"width":29,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":556,"width":28,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":679,"y":500,"width":21,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom7.jpg155011":{"filename":"classroom7.jpg","size":155011,"regions":[{"shape_attributes":{"name":"rect","x":45,"y":848,"width":65,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":219,"y":725,"width":59,"height":70},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":428,"y":699,"width":56,"height":60},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":589,"y":657,"width":46,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":612,"width":43,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":970,"y":539,"width":31,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1069,"y":520,"width":29,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":955,"y":473,"width":23,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":900,"y":501,"width":24,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":808,"y":527,"width":23,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":707,"y":530,"width":30,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":576,"y":570,"width":36,"height":45},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":131,"y":573,"width":33,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":185,"y":554,"width":25,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":291,"y":503,"width":23,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":245,"y":578,"width":29,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":556,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":604,"y":437,"width":14,"height":18},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":673,"y":489,"width":16,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom8.jpg149999":{"filename":"classroom8.jpg","size":149999,"regions":[{"shape_attributes":{"name":"rect","x":69,"y":826,"width":60,"height":64},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":391,"y":733,"width":57,"height":59},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":599,"y":621,"width":44,"height":60},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":603,"y":516,"width":43,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":804,"y":471,"width":29,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":767,"y":546,"width":40,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1059,"y":489,"width":26,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1145,"y":445,"width":18,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1162,"y":417,"width":25,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom9.jpg150320":{"filename":"classroom9.jpg","size":150320,"regions":[{"shape_attributes":{"name":"rect","x":52,"y":812,"width":64,"height":66},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":427,"y":722,"width":44,"height":64},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":570,"y":630,"width":58,"height":67},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":617,"y":521,"width":33,"height":46},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":793,"y":468,"width":29,"height":41},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":777,"y":558,"width":36,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1092,"y":497,"width":25,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1144,"y":442,"width":19,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":419,"width":23,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom10.jpg148598":{"filename":"classroom10.jpg","size":148598,"regions":[{"shape_attributes":{"name":"rect","x":63,"y":808,"width":64,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":417,"y":739,"width":55,"height":64},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":569,"y":620,"width":60,"height":67},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":613,"y":522,"width":36,"height":45},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":803,"y":470,"width":29,"height":41},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":772,"y":555,"width":38,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1174,"y":412,"width":21,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1104,"y":477,"width":26,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1075,"y":480,"width":22,"height":41},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom11.jpg148172":{"filename":"classroom11.jpg","size":148172,"regions":[{"shape_attributes":{"name":"rect","x":85,"y":839,"width":56,"height":58},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":416,"y":742,"width":58,"height":60},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":573,"y":615,"width":58,"height":64},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":604,"y":518,"width":36,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":801,"y":470,"width":30,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":554,"width":36,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1091,"y":490,"width":20,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1186,"y":415,"width":23,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1113,"y":465,"width":30,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom12.jpg147535":{"filename":"classroom12.jpg","size":147535,"regions":[{"shape_attributes":{"name":"rect","x":66,"y":828,"width":61,"height":63},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":413,"y":736,"width":55,"height":66},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":559,"y":596,"width":54,"height":69},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":522,"width":34,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":768,"y":554,"width":41,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":804,"y":471,"width":29,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1101,"y":500,"width":23,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":439,"width":19,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1189,"y":418,"width":24,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom13.jpg151516":{"filename":"classroom13.jpg","size":151516,"regions":[{"shape_attributes":{"name":"rect","x":63,"y":821,"width":66,"height":70},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":381,"y":736,"width":56,"height":57},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":530,"y":610,"width":59,"height":60},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":756,"y":559,"width":46,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":620,"y":519,"width":34,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":803,"y":467,"width":29,"height":42},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1098,"y":489,"width":25,"height":45},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1154,"y":435,"width":15,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1226,"y":426,"width":21,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom14.jpg148955":{"filename":"classroom14.jpg","size":148955,"regions":[{"shape_attributes":{"name":"rect","x":70,"y":813,"width":69,"height":77},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":710,"width":60,"height":64},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":615,"y":512,"width":39,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":810,"y":470,"width":27,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":757,"y":564,"width":45,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1099,"y":494,"width":25,"height":42},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1162,"y":423,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1150,"y":433,"width":17,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom15.jpg151769":{"filename":"classroom15.jpg","size":151769,"regions":[{"shape_attributes":{"name":"rect","x":53,"y":811,"width":43,"height":71},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":250,"y":773,"width":58,"height":81},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":486,"y":711,"width":52,"height":56},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":643,"width":49,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":273,"y":601,"width":18,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":380,"y":577,"width":27,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":621,"y":473,"width":12,"height":22},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":683,"y":493,"width":18,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":583,"y":585,"width":35,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":829,"y":635,"width":39,"height":42},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":765,"y":532,"width":26,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1133,"y":542,"width":28,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1097,"y":556,"width":17,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1009,"y":489,"width":20,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":885,"y":532,"width":22,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":923,"y":518,"width":22,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom16.jpg151539":{"filename":"classroom16.jpg","size":151539,"regions":[{"shape_attributes":{"name":"rect","x":130,"y":587,"width":31,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":176,"y":600,"width":24,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":213,"y":591,"width":29,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":374,"y":579,"width":31,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":681,"y":488,"width":21,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":624,"y":476,"width":15,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":15,"y":839,"width":61,"height":71},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":259,"y":799,"width":54,"height":64},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":490,"y":737,"width":51,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":600,"y":591,"width":34,"height":44},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":624,"y":648,"width":48,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":813,"y":621,"width":44,"height":52},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1131,"y":535,"width":29,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":775,"y":529,"width":27,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":876,"y":527,"width":22,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom17.jpg153007":{"filename":"classroom17.jpg","size":153007,"regions":[{"shape_attributes":{"name":"rect","x":170,"y":586,"width":23,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":206,"y":587,"width":32,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":382,"y":581,"width":28,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":81,"y":839,"width":43,"height":66},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":252,"y":798,"width":59,"height":65},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":595,"y":582,"width":33,"height":52},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":480,"y":729,"width":47,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":676,"y":647,"width":36,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":827,"y":628,"width":41,"height":44},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":674,"y":485,"width":17,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":552,"y":445,"width":19,"height":21},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":762,"y":523,"width":26,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":873,"y":533,"width":23,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":942,"y":534,"width":23,"height":21},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":982,"y":514,"width":27,"height":18},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1089,"y":550,"width":22,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1130,"y":539,"width":27,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom18.jpg149787":{"filename":"classroom18.jpg","size":149787,"regions":[{"shape_attributes":{"name":"rect","x":29,"y":849,"width":65,"height":72},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":267,"y":806,"width":63,"height":63},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":130,"y":590,"width":31,"height":42},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":218,"y":587,"width":26,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":161,"y":587,"width":21,"height":22},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":380,"y":579,"width":30,"height":21},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":598,"y":596,"width":31,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":679,"y":491,"width":16,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":956,"y":528,"width":21,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":882,"y":543,"width":23,"height":23},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":759,"y":524,"width":24,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":986,"y":498,"width":20,"height":21},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1071,"y":563,"width":27,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1139,"y":543,"width":25,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":846,"y":629,"width":45,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":633,"y":659,"width":46,"height":52},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":466,"y":718,"width":52,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom19.jpg150738":{"filename":"classroom19.jpg","size":150738,"regions":[{"shape_attributes":{"name":"rect","x":159,"y":593,"width":30,"height":42},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":222,"y":592,"width":31,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":25,"y":841,"width":53,"height":68},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":271,"y":818,"width":61,"height":52},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":389,"y":577,"width":30,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":450,"width":18,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":675,"y":488,"width":19,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":598,"y":595,"width":40,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":761,"y":524,"width":21,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":945,"y":529,"width":23,"height":23},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":845,"y":633,"width":42,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1127,"y":540,"width":28,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1070,"y":553,"width":27,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":661,"width":49,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":475,"y":719,"width":50,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom20.jpg147693":{"filename":"classroom20.jpg","size":147693,"regions":[{"shape_attributes":{"name":"rect","x":40,"y":815,"width":55,"height":83},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":124,"y":591,"width":32,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":167,"y":573,"width":21,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":224,"y":595,"width":24,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":385,"y":574,"width":24,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":282,"y":783,"width":53,"height":69},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":601,"y":657,"width":46,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":579,"y":588,"width":33,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":680,"y":482,"width":18,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":843,"y":630,"width":46,"height":46},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1076,"y":572,"width":26,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1142,"y":545,"width":23,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":962,"y":499,"width":23,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":815,"y":534,"width":22,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom21.jpg149411":{"filename":"classroom21.jpg","size":149411,"regions":[{"shape_attributes":{"name":"rect","x":66,"y":807,"width":58,"height":74},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":232,"y":746,"width":55,"height":75},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":128,"y":605,"width":27,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":176,"y":577,"width":25,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":210,"y":588,"width":31,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":639,"y":656,"width":51,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":439,"y":690,"width":51,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":682,"y":490,"width":20,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":723,"y":523,"width":32,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":792,"y":610,"width":40,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1218,"y":480,"width":44,"height":69},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1091,"y":522,"width":28,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1000,"y":540,"width":27,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":925,"y":494,"width":28,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":869,"y":518,"width":28,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":829,"y":525,"width":18,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":381,"y":577,"width":34,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom22.jpg149420":{"filename":"classroom22.jpg","size":149420,"regions":[{"shape_attributes":{"name":"rect","x":61,"y":819,"width":59,"height":75},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":127,"y":587,"width":30,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":163,"y":571,"width":27,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":297,"y":542,"width":24,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":221,"y":584,"width":32,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":387,"y":577,"width":26,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":610,"y":430,"width":17,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":670,"y":495,"width":21,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":575,"y":569,"width":36,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":623,"y":657,"width":45,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":813,"y":525,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1022,"y":565,"width":27,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1079,"y":515,"width":25,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":932,"y":493,"width":23,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":880,"y":515,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom23.jpg148286":{"filename":"classroom23.jpg","size":148286,"regions":[{"shape_attributes":{"name":"rect","x":19,"y":578,"width":30,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":188,"y":544,"width":34,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":328,"y":500,"width":29,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":351,"y":568,"width":36,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":94,"y":610,"width":43,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":448,"y":420,"width":23,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":587,"y":409,"width":14,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":658,"y":441,"width":25,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":844,"y":470,"width":18,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":873,"y":426,"width":25,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1041,"y":488,"width":15,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":890,"y":582,"width":42,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":525,"y":509,"width":32,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":580,"y":614,"width":48,"height":54},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":363,"y":701,"width":39,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom24.jpg146985":{"filename":"classroom24.jpg","size":146985,"regions":[{"shape_attributes":{"name":"rect","x":24,"y":566,"width":30,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":169,"y":533,"width":32,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":317,"y":501,"width":29,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":88,"y":600,"width":45,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":338,"y":680,"width":51,"height":60},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":329,"y":577,"width":42,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":452,"y":426,"width":19,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":568,"y":411,"width":22,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":646,"y":434,"width":27,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":524,"y":513,"width":32,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1232,"y":388,"width":39,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":884,"y":428,"width":24,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":785,"y":467,"width":29,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":866,"y":550,"width":50,"height":57},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":591,"y":619,"width":52,"height":54},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom25.jpg145925":{"filename":"classroom25.jpg","size":145925,"regions":[{"shape_attributes":{"name":"rect","x":23,"y":555,"width":30,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":170,"y":533,"width":30,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":93,"y":592,"width":41,"height":58},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":319,"y":502,"width":27,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":359,"y":575,"width":40,"height":46},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":456,"y":429,"width":22,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":624,"y":439,"width":22,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1168,"y":393,"width":43,"height":59},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1010,"y":492,"width":33,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":872,"y":434,"width":23,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":791,"y":468,"width":25,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":847,"y":476,"width":20,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":882,"y":551,"width":43,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":647,"y":612,"width":43,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":519,"y":514,"width":36,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":317,"y":686,"width":56,"height":67},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom26.jpg151957":{"filename":"classroom26.jpg","size":151957,"regions":[{"shape_attributes":{"name":"rect","x":12,"y":555,"width":32,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":165,"y":541,"width":31,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":91,"y":597,"width":41,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":436,"y":432,"width":23,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":574,"y":441,"width":25,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":882,"y":426,"width":20,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":793,"y":464,"width":22,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1203,"y":405,"width":23,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1082,"y":495,"width":30,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":883,"y":575,"width":41,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":346,"y":515,"width":26,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":378,"y":579,"width":37,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":530,"y":505,"width":31,"height":42},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":590,"y":593,"width":49,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":152,"y":694,"width":51,"height":69},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom27.jpg152331":{"filename":"classroom27.jpg","size":152331,"regions":[{"shape_attributes":{"name":"rect","x":17,"y":558,"width":29,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":96,"y":595,"width":40,"height":52},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":327,"y":488,"width":29,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":175,"y":545,"width":26,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":439,"y":433,"width":22,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":576,"y":440,"width":26,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":868,"y":427,"width":25,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":796,"y":473,"width":22,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1150,"y":405,"width":24,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1022,"y":479,"width":28,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":883,"y":547,"width":45,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":585,"y":606,"width":49,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":374,"y":578,"width":42,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":546,"y":500,"width":31,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":143,"y":694,"width":55,"height":69},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom28.jpg154093":{"filename":"classroom28.jpg","size":154093,"regions":[{"shape_attributes":{"name":"rect","x":20,"y":551,"width":33,"height":45},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":322,"y":506,"width":31,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":189,"y":541,"width":33,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":91,"y":594,"width":44,"height":54},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":431,"y":431,"width":24,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":559,"y":431,"width":21,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":691,"y":408,"width":20,"height":21},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":519,"y":511,"width":36,"height":44},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":623,"y":438,"width":27,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":797,"y":473,"width":26,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1144,"y":405,"width":25,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1244,"y":379,"width":23,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1006,"y":479,"width":32,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":866,"y":431,"width":26,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":905,"y":581,"width":46,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":579,"y":610,"width":53,"height":56},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":138,"y":707,"width":58,"height":60},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,
{"classroom29.jpg153166":{"filename":"classroom29.jpg","size":153166,"regions":[{"shape_attributes":{"name":"rect","x":151,"y":701,"width":43,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":90,"y":594,"width":46,"height":52},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":21,"y":556,"width":28,"height":41},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":191,"y":546,"width":35,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":334,"y":497,"width":26,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":506,"y":516,"width":38,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":433,"y":433,"width":23,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":561,"y":435,"width":16,"height":21},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":661,"y":438,"width":23,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":586,"y":607,"width":50,"height":54},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":900,"y":590,"width":40,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1142,"y":406,"width":26,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1021,"y":482,"width":27,"height":37},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":863,"y":430,"width":29,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom30.jpg149988":{"filename":"classroom30.jpg","size":149988,"regions":[{"shape_attributes":{"name":"rect","x":20,"y":554,"width":33,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":191,"y":543,"width":31,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":305,"y":499,"width":33,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":92,"y":599,"width":42,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":510,"y":516,"width":28,"height":46},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":345,"y":550,"width":31,"height":52},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":569,"y":610,"width":48,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":696,"y":651,"width":152,"height":169},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":883,"y":548,"width":47,"height":59},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":140,"y":710,"width":54,"height":62},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":461,"y":435,"width":22,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":560,"y":434,"width":18,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":661,"y":437,"width":24,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":870,"y":429,"width":21,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":776,"y":471,"width":23,"height":25},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1035,"y":482,"width":19,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1216,"y":415,"width":21,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
]

m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in groundTruth:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredgroundTruthObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredgroundTruthObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        tuplesnthPhoto.append((region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1

In [ ]:
i=0
TPTotal=0
TNTotal=0
FPTotal=0
FNTotal=0

for element in tuples:
    TP=0
    TN=0
    FN=0
    FP=0
    verifiedPredictions=0    
    print(f"------Results for image {i+1}------")
    for face in element: #Compare each face of ground truth with the predictions' array. If found, add TP, if not identified, add FN
        found=False
        for detectedFace in boundingBoxDetectedFaces[i]:
            IoU=calculate_iou_detection(face, detectedFace)
            if (IoU>=0.5):
                found=True
                TP=TP+1
                verifiedPredictions=verifiedPredictions+1
                break
        if (found==False):
            FN=FN+1
    FP=FP+(len(boundingBoxDetectedFaces[i])-verifiedPredictions)
    print(F"TP: {TP}, TN:{TN}, FP:{FP} FN:{FN}")
    TPTotal=TPTotal+TP
    TNTotal=TNTotal+TN
    FPTotal=FPTotal+FP
    FNTotal=FNTotal+FN


    i=i+1


print("----Total----")
print(f"TP: {TPTotal}, TN: {TNTotal}, FP:{FPTotal},FN: {FNTotal}")      



Take all images of the folder and all student's personal image and draw a unique color rectangle if recognized

In [ ]:
colors = [
    (255, 0, 0),     
    (0, 255, 0),     
    (0, 0, 255),     
    (255, 255, 0),   
    (255, 0, 255),   
    (0, 255, 255),  
    (128, 0, 0),     
    (0, 128, 0),     
    (0, 0, 128),      
    (128, 128, 0),    
    (128, 0, 128),    
    (0, 128, 128),    
    (64, 0, 128),     
    (128, 128, 255),  
    (0, 255, 128),   
    (128, 255, 128),  
    (192, 64, 64),   
    (64, 192, 192),  
    (255, 128, 192), 
    (32, 64, 255)    
]
#Array that contains all faces recognized in all test images. Each photo corresponds to the n-th array element
boundingBoxRecognizedFaces=[]
recognized_counter_array=[]
m=1
photo_index=1

#Create image of the class with all faces recognized in it
for file in os.listdir(classroomFolder):
    camera_local_file = os.path.join("..","ImagesRekognition","RealClassroomPhotos", file)
    print(f"Using index {photo_index}")

    if (photo_index>=1 and photo_index<=2) or  (photo_index>=23 and photo_index<=30):
        selectedCourse="StudentsRobotics"
        student_array=["AlbaQuiroz", "AlejandroCasallas","AlejandroVelasco","AlexanderDominguez","AndresAnillo","CarlosBornacelly","DavidChaparro"
                    ,"EstebanToro","GabrielCastanez","GiohanOlivares","JesusCotes","JoseVilla","JuanMolina" ,"ManuelRangel","MariaBerdugo","MariaLopez",
                    "MauricioDeLaHoz", "SergioFonseca","YordiRochel"]    
    elif (photo_index>=3 and photo_index<=5) or (photo_index>=8 and photo_index<=14):
        selectedCourse="StudentsElectronicDesign1"
        student_array=["AllysonSalom", "AngieValencia","DanielDelgado","DiegoGomez", "DylanDeOro","FelipeOspino","HabidAbdala","JuanLozano","WilmanDaza"]

    elif (photo_index>=6 and photo_index<=7) or  (photo_index>=15 and photo_index<=22):
        selectedCourse="StudentsElectronicDesign2"
        student_array=["AliRada","AndreaQuintero","AndresFabregas","AndresNarvaez","BrayanRubiano","DiegoGomez","EdwinUtria","GabrielaBecerra",
                    "JuanHernandez","JuanQuintero","JuanSanchez","LucianaDeLaRosa","MariaFerrer","MayraCarreno",
                    "OctavioMorales","SamirBarcelo","SteevenBallena"]

    #convert files to bytes in order to be able to read it
    with open(camera_local_file, 'rb') as image_file:
        camera_file_bytes = image_file.read()
    responseDetection=client.detect_faces( Image={'Bytes': camera_file_bytes})
    image = cv2.imread(camera_local_file)

    k=0
    recognized_counter=0

    boundingBoxRecognizedFacesithImage=[]

    for student_name in student_array:
        student_found = False  # Bandera para controlar si el estudiante fue encontrado
        
        student_id_photo=os.path.join("..","ImagesRekognition",selectedCourse, f"{student_name}.jpeg")
        with open(student_id_photo, 'rb') as image_file:
            student_id_photo_bytes = image_file.read()

        #Look for the face in SourceImage evaluating all the faces in TargetImage
        responseRecognition=client.compare_faces(
            SourceImage={'Bytes': student_id_photo_bytes},
            TargetImage={'Bytes': camera_file_bytes},
            SimilarityThreshold=80
        )
        # Read the target image with OpenCV

        # Get image dimensions
        image_height, image_width, _ = image.shape

        # Draw bounding boxes for matched faces
        for faceMatch in responseRecognition['FaceMatches']:
            # Get bounding box coordinates
            bbox = faceMatch['Face']['BoundingBox']
            
            # Convert normalized coordinates to pixel coordinates
            left = int(bbox['Left'] * image_width)
            top = int(bbox['Top'] * image_height)
            width = int(bbox['Width'] * image_width)
            height = int(bbox['Height'] * image_height)
        
            # Draw green rectangle (to differentiate from previous red detection boxes)
            cv2.rectangle(
                image, 
                (left, top), 
                (left + width, top + height), 
                colors[k],  # Green color in BGR
                2,
            )
            
            padding = 5  # Espacio entre el rectángulo y el cuadro de texto
            text_box_height = 25  # Altura del cuadro de texto
            # Agregar texto dentro del recuadro de texto
            cv2.putText(image, f"{student_name}", (left + 10, top - padding - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[k], 1)

            recognized_counter=recognized_counter+1
            print('Face at coordinates ' + str(bbox) + ' matches with ' + str(faceMatch['Similarity']) + '% confidence'+ f"and is likely {student_name}")

            #Save the identity and region as a tuple in the n-th array element that corresponds to the n-th image
            boundingBoxRecognizedFacesithImage.append((student_name,left,top,width,height))
            student_found = True  # Marcamos que este estudiante fue encontrado
        
        # Si no se encontró al estudiante, agregar una tupla con valores cero
        if not student_found:
            boundingBoxRecognizedFacesithImage.append((student_name, 0, 0, 0, 0))
            print(f"Student {student_name} not found in the image")
            
        k=k-1
    boundingBoxRecognizedFaces.append(boundingBoxRecognizedFacesithImage)
    recognized_counter_array.append(recognized_counter)
    # Save the annotated image
    output_path = os.path.join("..","ImagesRekognition","StudentFoundInClassroom", f"faces_in_classroom_{m}_{recognized_counter}.jpeg")
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")
    m=m+1
    photo_index=photo_index+1

Obtain tuples from base line (Manual correction and verification)

In [34]:
true=True
faceGallery=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":348,"width":21,"height":27},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":733,"y":402,"width":29,"height":34},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":485,"width":38,"height":44},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":337,"width":20,"height":23},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":164,"y":481,"width":35,"height":38},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":567,"y":548,"width":57,"height":58},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":403,"y":582,"width":59,"height":78},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":332,"width":20,"height":22},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":15,"y":506,"width":35,"height":40},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1189,"y":330,"width":31,"height":31},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":122,"y":538,"width":38,"height":55},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":440,"y":368,"width":24,"height":26},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":806,"y":460,"width":47,"height":55},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":366,"width":26,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":399,"width":30,"height":37},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":717,"y":350,"width":22,"height":27},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":547,"y":431,"width":37,"height":44},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":372,"y":427,"width":28,"height":33},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom2.jpeg152579":{"filename":"classroom2.jpeg","size":152579,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":347,"width":25,"height":28},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":729,"y":394,"width":31,"height":35},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":340,"y":481,"width":37,"height":46},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":617,"y":336,"width":20,"height":25},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":160,"y":481,"width":31,"height":37},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":580,"y":541,"width":55,"height":65},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":428,"y":580,"width":51,"height":76},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":333,"width":20,"height":20},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":28,"y":504,"width":29,"height":38},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1185,"y":328,"width":27,"height":32},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":171,"y":538,"width":40,"height":57},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":443,"y":366,"width":23,"height":27},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":809,"y":459,"width":46,"height":53},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1126,"y":375,"width":25,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1039,"y":395,"width":25,"height":36},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":720,"y":350,"width":20,"height":30},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":542,"y":428,"width":38,"height":43},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":373,"y":425,"width":27,"height":32},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom3.jpg155001":{"filename":"classroom3.jpg","size":155001,"regions":[{"shape_attributes":{"name":"rect","x":526,"y":587,"width":59,"height":70},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":605,"y":499,"width":37,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":537,"width":43,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":31,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":408,"width":23,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1107,"y":430,"width":26,"height":32},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1026,"y":479,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom4.jpg156291":{"filename":"classroom4.jpg","size":156291,"regions":[{"shape_attributes":{"name":"rect","x":552,"y":592,"width":48,"height":62},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":500,"width":35,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":538,"width":39,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":453,"width":28,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":480,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":25,"height":33},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom5.jpg156524":{"filename":"classroom5.jpg","size":156524,"regions":[{"shape_attributes":{"name":"rect","x":551,"y":592,"width":49,"height":59},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":769,"y":540,"width":42,"height":49},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":500,"width":36,"height":45},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":29,"height":41},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1020,"y":480,"width":33,"height":39},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":26,"height":34},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom6.jpg153504":{"filename":"classroom6.jpg","size":153504,"regions":[{"shape_attributes":{"name":"rect","x":59,"y":845,"width":67,"height":74},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":221,"y":721,"width":58,"height":72},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":434,"y":701,"width":50,"height":60},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":584,"y":657,"width":47,"height":47},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":772,"y":616,"width":42,"height":45},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":976,"y":539,"width":29,"height":37},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1077,"y":516,"width":26,"height":30},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":957,"y":472,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":903,"y":501,"width":23,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":811,"y":526,"width":21,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":704,"y":532,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":574,"y":567,"width":40,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":29},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":243,"y":581,"width":30,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":520,"width":21,"height":26},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":193,"y":553,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":155,"y":576,"width":31,"height":36},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom7.jpg155011":{"filename":"classroom7.jpg","size":155011,"regions":[{"shape_attributes":{"name":"rect","x":46,"y":849,"width":64,"height":79},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":219,"y":724,"width":58,"height":71},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":430,"y":696,"width":55,"height":65},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":589,"y":661,"width":46,"height":49},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":612,"width":44,"height":54},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":969,"y":539,"width":32,"height":39},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1069,"y":518,"width":29,"height":33},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":954,"y":475,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":898,"y":501,"width":27,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":807,"y":527,"width":24,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":575,"y":569,"width":38,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":706,"y":531,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":31},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":505,"width":24,"height":29},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":244,"y":579,"width":28,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":186,"y":555,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":134,"y":574,"width":31,"height":38},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom8.jpg149999":{"filename":"classroom8.jpg","size":149999,"regions":[{"shape_attributes":{"name":"rect","x":63,"y":822,"width":64,"height":66},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":386,"y":722,"width":61,"height":67},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":593,"y":618,"width":50,"height":62},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":516,"width":33,"height":54},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":802,"y":471,"width":32,"height":42},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":764,"y":546,"width":44,"height":49},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1058,"y":489,"width":28,"height":36},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1161,"y":418,"width":27,"height":29},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom9.jpg150320":{"filename":"classroom9.jpg","size":150320,"regions":[{"shape_attributes":{"name":"rect","x":52,"y":814,"width":66,"height":65},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":429,"y":726,"width":45,"height":60},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":571,"y":628,"width":57,"height":70},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":517,"width":37,"height":53},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":773,"y":552,"width":39,"height":53},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":794,"y":465,"width":30,"height":44},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1089,"y":495,"width":27,"height":39},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom10.jpg148598":{"filename":"classroom10.jpg","size":148598,"regions":[{"shape_attributes":{"name":"rect","x":64,"y":814,"width":63,"height":70},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":573,"y":621,"width":56,"height":67},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":413,"y":735,"width":58,"height":66},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":773,"y":550,"width":38,"height":55},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":610,"y":520,"width":38,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":805,"y":474,"width":30,"height":36},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1103,"y":479,"width":30,"height":32},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1171,"y":411,"width":25,"height":36},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom11.jpg148172":{"filename":"classroom11.jpg","size":148172,"regions":[{"shape_attributes":{"name":"rect","x":75,"y":819,"width":67,"height":77},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":573,"y":611,"width":57,"height":70},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":410,"y":734,"width":59,"height":68},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":766,"y":549,"width":44,"height":58},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":604,"y":514,"width":36,"height":57},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":800,"y":472,"width":31,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1115,"y":465,"width":29,"height":41},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1184,"y":413,"width":26,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom12.jpg147535":{"filename":"classroom12.jpg","size":147535,"regions":[{"shape_attributes":{"name":"rect","x":555,"y":595,"width":57,"height":68},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":59,"y":826,"width":70,"height":65},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":407,"y":732,"width":63,"height":74},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":521,"width":37,"height":51},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":804,"y":470,"width":30,"height":44},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":553,"width":38,"height":48},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1104,"y":504,"width":22,"height":40},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom13.jpg151516":{"filename":"classroom13.jpg","size":151516,"regions":[{"shape_attributes":{"name":"rect","x":52,"y":818,"width":75,"height":72},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":377,"y":734,"width":59,"height":57},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":530,"y":604,"width":58,"height":70},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":618,"y":521,"width":35,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":802,"y":470,"width":31,"height":40},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":758,"y":558,"width":42,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1099,"y":494,"width":26,"height":39},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom14.jpg148955":{"filename":"classroom14.jpg","size":148955,"regions":[{"shape_attributes":{"name":"rect","x":75,"y":815,"width":63,"height":71},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":346,"y":712,"width":56,"height":61},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":617,"y":513,"width":37,"height":53},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":757,"y":567,"width":50,"height":50},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":810,"y":467,"width":28,"height":43},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1100,"y":498,"width":24,"height":36},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1163,"y":424,"width":25,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom15.jpg151769":{"filename":"classroom15.jpg","size":151769,"regions":[{"shape_attributes":{"name":"rect","x":51,"y":814,"width":47,"height":69},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":248,"y":779,"width":63,"height":74},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":270,"y":598,"width":28,"height":39},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":379,"y":577,"width":28,"height":28},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":581,"y":583,"width":36,"height":48},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":482,"y":711,"width":54,"height":57},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":642,"width":49,"height":54},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":999,"y":486,"width":29,"height":31},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":766,"y":529,"width":27,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1127,"y":541,"width":32,"height":38},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom16.jpg151539":{"filename":"classroom16.jpg","size":151539,"regions":[{"shape_attributes":{"name":"rect","x":127,"y":588,"width":36,"height":39},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":21,"y":835,"width":56,"height":72},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":253,"y":795,"width":57,"height":71},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":487,"y":733,"width":54,"height":57},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":620,"y":646,"width":50,"height":55},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":377,"y":577,"width":29,"height":31},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":177,"y":597,"width":24,"height":30},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":596,"y":582,"width":39,"height":50},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":873,"y":524,"width":25,"height":34},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":774,"y":532,"width":29,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom17.jpg153007":{"filename":"classroom17.jpg","size":153007,"regions":[{"shape_attributes":{"name":"rect","x":248,"y":793,"width":67,"height":70},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":870,"y":527,"width":25,"height":34},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":672,"y":640,"width":40,"height":56},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":590,"y":582,"width":39,"height":53},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":208,"y":586,"width":30,"height":35},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":471,"y":725,"width":55,"height":49},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom18.jpg149787":{"filename":"classroom18.jpg","size":149787,"regions":[{"shape_attributes":{"name":"rect","x":213,"y":582,"width":33,"height":40},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":383,"y":576,"width":27,"height":30},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":31,"y":852,"width":61,"height":70},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":274,"y":800,"width":56,"height":70},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":632,"y":656,"width":50,"height":57},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":592,"y":590,"width":40,"height":50},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":760,"y":521,"width":27,"height":38},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1069,"y":559,"width":32,"height":37},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom19.jpg150738":{"filename":"classroom19.jpg","size":150738,"regions":[{"shape_attributes":{"name":"rect","x":21,"y":837,"width":59,"height":71},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":273,"y":817,"width":58,"height":53},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":608,"y":662,"width":53,"height":50},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":227,"y":591,"width":27,"height":30},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":597,"y":593,"width":41,"height":46},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1068,"y":551,"width":30,"height":41},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":756,"y":524,"width":29,"height":39},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom20.jpg147693":{"filename":"classroom20.jpg","size":147693,"regions":[{"shape_attributes":{"name":"rect","x":43,"y":818,"width":50,"height":80},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":786,"width":46,"height":64},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":597,"y":656,"width":53,"height":49},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":577,"y":582,"width":37,"height":50},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":124,"y":593,"width":34,"height":34},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1078,"y":572,"width":24,"height":35},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":960,"y":498,"width":25,"height":29},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":162,"y":573,"width":23,"height":30},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom21.jpg149411":{"filename":"classroom21.jpg","size":149411,"regions":[{"shape_attributes":{"name":"rect","x":75,"y":807,"width":49,"height":74},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":242,"y":739,"width":46,"height":75},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":440,"y":693,"width":53,"height":45},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":639,"y":662,"width":50,"height":46},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":790,"y":606,"width":44,"height":49},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":124,"y":606,"width":33,"height":40},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":213,"y":586,"width":27,"height":34},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":996,"y":538,"width":31,"height":39},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1227,"y":477,"width":34,"height":72},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1092,"y":519,"width":28,"height":39},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":926,"y":493,"width":24,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":871,"y":517,"width":26,"height":30},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":724,"y":521,"width":30,"height":40},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom22.jpg149420":{"filename":"classroom22.jpg","size":149420,"regions":[{"shape_attributes":{"name":"rect","x":127,"y":589,"width":31,"height":38},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":221,"y":586,"width":28,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":164,"y":568,"width":24,"height":31},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":69,"y":819,"width":51,"height":74},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":572,"y":564,"width":39,"height":54},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1082,"y":516,"width":24,"height":34},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":810,"y":523,"width":27,"height":33},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":933,"y":490,"width":23,"height":32},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":879,"y":514,"width":26,"height":31},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":623,"y":656,"width":44,"height":49},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom23.jpg148286":{"filename":"classroom23.jpg","size":148286,"regions":[{"shape_attributes":{"name":"rect","x":96,"y":610,"width":44,"height":46},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":583,"y":609,"width":49,"height":61},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":187,"y":539,"width":35,"height":34},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":329,"y":499,"width":29,"height":36},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":524,"y":511,"width":37,"height":41},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":349,"y":569,"width":36,"height":51},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":449,"y":422,"width":22,"height":23},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":656,"y":439,"width":28,"height":32},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":893,"y":577,"width":43,"height":48},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom24.jpg146985":{"filename":"classroom24.jpg","size":146985,"regions":[{"shape_attributes":{"name":"rect","x":87,"y":596,"width":47,"height":55},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":317,"y":502,"width":28,"height":37},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":170,"y":535,"width":31,"height":36},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":22,"y":563,"width":32,"height":41},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":332,"y":576,"width":39,"height":48},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":594,"y":618,"width":47,"height":56},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":340,"y":678,"width":49,"height":63},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":526,"y":509,"width":32,"height":43},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":870,"y":550,"width":47,"height":57},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":568,"y":412,"width":21,"height":26},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":646,"y":432,"width":27,"height":32},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":451,"y":426,"width":20,"height":26},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":785,"y":466,"width":27,"height":31},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom25.jpg145925":{"filename":"classroom25.jpg","size":145925,"regions":[{"shape_attributes":{"name":"rect","x":92,"y":595,"width":43,"height":57},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":169,"y":534,"width":32,"height":35},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":320,"y":502,"width":28,"height":35},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":319,"y":683,"width":52,"height":70},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":22,"y":555,"width":30,"height":41},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":648,"y":608,"width":42,"height":54},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":883,"y":544,"width":44,"height":58},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1010,"y":487,"width":31,"height":41},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":791,"y":469,"width":26,"height":30},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":456,"y":430,"width":21,"height":25},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":517,"y":510,"width":37,"height":46},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":361,"y":575,"width":37,"height":51},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":619,"y":432,"width":28,"height":36},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom26.jpg151957":{"filename":"classroom26.jpg","size":151957,"regions":[{"shape_attributes":{"name":"rect","x":143,"y":697,"width":60,"height":68},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":586,"y":594,"width":54,"height":56},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":883,"y":569,"width":47,"height":50},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1085,"y":493,"width":30,"height":41},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1202,"y":405,"width":26,"height":33},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":88,"y":593,"width":47,"height":54},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":371,"y":575,"width":49,"height":54},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":530,"y":499,"width":33,"height":49},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":793,"y":460,"width":24,"height":35},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":163,"y":540,"width":34,"height":36},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":9,"y":555,"width":35,"height":41},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":340,"y":510,"width":32,"height":36},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":571,"y":437,"width":27,"height":36},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":434,"y":430,"width":25,"height":30},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom27.jpg152331":{"filename":"classroom27.jpg","size":152331,"regions":[{"shape_attributes":{"name":"rect","x":146,"y":695,"width":53,"height":66},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":583,"y":605,"width":52,"height":56},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":883,"y":540,"width":49,"height":57},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1020,"y":478,"width":31,"height":39},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1149,"y":407,"width":27,"height":34},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":790,"y":471,"width":29,"height":31},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":543,"y":500,"width":33,"height":44},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":374,"y":576,"width":44,"height":50},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":93,"y":595,"width":42,"height":52},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":16,"y":555,"width":31,"height":42},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":329,"y":489,"width":29,"height":38},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":574,"y":441,"width":30,"height":33},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":437,"y":432,"width":24,"height":29},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom28.jpg154093":{"filename":"classroom28.jpg","size":154093,"regions":[{"shape_attributes":{"name":"rect","x":22,"y":550,"width":31,"height":47},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":326,"y":507,"width":28,"height":35},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":136,"y":699,"width":56,"height":66},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":92,"y":592,"width":45,"height":55},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":584,"y":607,"width":50,"height":60},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":515,"y":510,"width":42,"height":48},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1145,"y":404,"width":25,"height":35},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1006,"y":479,"width":33,"height":38},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":798,"y":471,"width":28,"height":35},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":625,"y":438,"width":25,"height":33},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":431,"y":432,"width":23,"height":28},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":901,"y":583,"width":50,"height":47},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom29.jpg153166":{"filename":"classroom29.jpg","size":153166,"regions":[{"shape_attributes":{"name":"rect","x":91,"y":593,"width":45,"height":51},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":330,"y":493,"width":30,"height":35},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":192,"y":544,"width":31,"height":30},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":435,"y":432,"width":23,"height":26},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":504,"y":513,"width":40,"height":47},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":146,"y":695,"width":50,"height":66},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":17,"y":556,"width":34,"height":44},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":587,"y":604,"width":50,"height":62},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1015,"y":479,"width":32,"height":40},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1140,"y":403,"width":29,"height":36},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":654,"y":437,"width":29,"height":34},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom30.jpg149988":{"filename":"classroom30.jpg","size":149988,"regions":[{"shape_attributes":{"name":"rect","x":880,"y":541,"width":49,"height":69},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":572,"y":607,"width":51,"height":59},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":142,"y":706,"width":53,"height":64},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":96,"y":600,"width":39,"height":54},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":306,"y":497,"width":32,"height":36},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":506,"y":514,"width":38,"height":49},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":188,"y":539,"width":34,"height":34},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":17,"y":551,"width":35,"height":41},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
]

m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in faceGallery:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        #print(f"x = {region["x"]} \ny = {region["y"]} \nwidth = {region["width"]}\nheight = {region["height"]} \nidentifier = {identity["name"]}")
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1


Code to calculate IoU

In [32]:

def calculate_iou(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    # Extraer coordenadas
    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    # Calcular coordenadas de la intersección
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    # Comprobar si hay intersección
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    # Calcular áreas
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    # Calcular unión
    area_union = area_box1 + area_box2 - area_intersect
    
    # Calcular IoU
    iou = area_intersect / area_union
    
    return iou


Iterate through the base line and compare it to the results obtained by Rekognition. Then calculate TP, TN, FP, FN

In [37]:
#It searchs based on face gallery/Base Line: It compares my definition of the existance of specific students and compares it to the found studens
#The result wouldn't be affected if the search direction is reversed

i=0
TPTotal=0
TNTotal=0
FPTotal=0
FNTotal=0
#Loop through all the photos
for photoObject in tuples:
    print(f"Results for photo {i+1}")
    TP=0
    TN=0
    FP=0
    FN=0
    #Obtain the array of faces recognized with Rekognition in that photo
    tuplesToCompare=boundingBoxRecognizedFaces[i]

    listOfNamesBaseline=[]
    for element in photoObject:
        listOfNamesBaseline.append(element[0])

    listOfNamesModel=[]
    for element in tuplesToCompare:
        listOfNamesModel.append(element[0])


    for element in photoObject:
        if(element[0] in listOfNamesModel):
            for identity in tuplesToCompare:
                IoU=calculate_iou(element,identity)

                #Case True Positive: Identity and bounding box match
                if(IoU>=0.5 and element[0]==identity[0]):
                    TP=TP+1

                #Case False Positive: no identity match, however, the bounding boxes are similar
                if(IoU>=0.5 and element[0]!=identity[0]):
                    FP=FP+1

                #Case True Negative: No bounding box match. However, the identity is different
                if(IoU<0.5 and element[0]!=identity[0]):
                    TN=TN+1
                #Case False Negative: The identity matches. However, bonding boxes do not correspond
                if(IoU<0.5 and element[0]==identity[0]):
                    FN=FN+1 
            
            #Correction to take into account if the model doesn't mistake identities of strangers: Detected Faces-Faces identified        
            TN=TN+len(boundingBoxDetectedFaces[i])-len(listOfNamesModel)
        else:
            FN=FN+1
            TN=TN+len(boundingBoxDetectedFaces[i])-1
            print(f"{element[0]} not in model's list and adds 1 FN and {len(boundingBoxDetectedFaces)-1} TN")
            
    TPTotal=TPTotal+TP
    TNTotal=TNTotal+TN
    FPTotal=FPTotal+FP
    FNTotal=FNTotal+FN
  
    #print(recognized_counter_array[i])
    print(f"TP : {TP} \nTN : {TN} \nFP : {FP} \nFN : {FN}")
    i=i+1

print("----Total----")
print(f"TP: {TPTotal}, TN: {TNTotal}, FP:{FPTotal},FN: {FNTotal}") 

Results for photo 1
TP : 17 
TN : 360 
FP : 0 
FN : 1
Results for photo 2
TP : 14 
TN : 360 
FP : 0 
FN : 4
Results for photo 3
TP : 6 
TN : 56 
FP : 0 
FN : 1
Results for photo 4
TP : 7 
TN : 56 
FP : 0 
FN : 0
Results for photo 5
TP : 7 
TN : 56 
FP : 0 
FN : 0
Results for photo 6
TP : 14 
TN : 289 
FP : 0 
FN : 3
Results for photo 7
TP : 14 
TN : 306 
FP : 0 
FN : 3
Results for photo 8
TP : 7 
TN : 64 
FP : 0 
FN : 1
Results for photo 9
TP : 6 
TN : 56 
FP : 0 
FN : 1
Results for photo 10
TP : 7 
TN : 64 
FP : 0 
FN : 1
Results for photo 11
TP : 7 
TN : 64 
FP : 0 
FN : 1
Results for photo 12
TP : 7 
TN : 56 
FP : 0 
FN : 0
Results for photo 13
TP : 6 
TN : 56 
FP : 0 
FN : 1
Results for photo 14
TP : 5 
TN : 56 
FP : 0 
FN : 2
Results for photo 15
TP : 6 
TN : 170 
FP : 0 
FN : 4
Results for photo 16
TP : 6 
TN : 150 
FP : 0 
FN : 4
Results for photo 17
TP : 4 
TN : 96 
FP : 0 
FN : 2
Results for photo 18
TP : 6 
TN : 128 
FP : 0 
FN : 2
Results for photo 19
TP : 5 
TN : 98 
FP : 0